# Оптимизация выполнения кода, векторизация, Numba

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

## Лабораторная работа 3

In [44]:
import numpy as np
import pandas as pd
from numba import jit, njit
import numba
from typing import Union

%load_ext line_profiler

## Измерение времени выполнения кода

Назовем полным описанием рецепта строку, полученную путем конкатенации названия и описания рецепта через пробел. Удалите строки для рецептов, которые были добавлены не в 2010 году.

Реализуйте несколько вариантов функции подсчета средней длины полного описания рецепта для рецептов, добавленных в 2010 году.

In [121]:
recipes = pd.read_csv('recipes_sample.csv', sep = ',')
reviews = pd.read_csv('reviews_sample.csv', sep = ',', index_col = 0)

In [122]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])

In [123]:
recipes_2010 = recipes[recipes['submitted'].dt.year == 2010]
recipes_2010.head()

,name,id,minutes,contributor_id,submitted,n_steps,description,n_ingredients
52,just peachy cobbler,437637,70,1085867,2010-09-17,10.0,all i can say is yummmmmm . . . a simple to ma...,10.0
68,the heat spicy party mix,437219,95,1682162,2010-09-13,NaN,a spicy chex mix that will really warm your gu...,11.0
81,iowa state fair sweet dough caramel cinnamon ...,435816,80,17803,2010-08-24,29.0,this was the winning entry at the 2010 iowa st...,NaN
104,1 minute blueberries cream,428566,2,1375473,2010-06-04,4.0,i was craving blueberry tonight but wanted non...,NaN
146,2 2 2 diet mocha,416599,5,789314,2010-03-15,5.0,"while trying to come up with a satisfying ""sna...",7.0


In [124]:
recipes['submitted'] = pd.to_datetime(recipes['submitted'])
recipes = recipes[recipes['submitted'].dt.year == 2010]

In [37]:
recipes_2010['full_description'] = recipes_2010.name + ' ' + recipes_2010.description

/var/folders/0f/kzrl9rx52273k8v4glcycjv80000gn/T/ipykernel_77846/3410906728.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  recipes_2010['full_description'] = recipes_2010.name + ' ' + recipes_2010.description


In [38]:
recipes_2010 = recipes_2010.drop(columns = ['name', 'description'])
recipes_2010.head()

,id,minutes,contributor_id,submitted,n_steps,n_ingredients,full_description
52,437637,70,1085867,2010-09-17,10.0,10.0,just peachy cobbler all i can say is yummmmmm...
68,437219,95,1682162,2010-09-13,NaN,11.0,the heat spicy party mix a spicy chex mix tha...
81,435816,80,17803,2010-08-24,29.0,NaN,iowa state fair sweet dough caramel cinnamon ...
104,428566,2,1375473,2010-06-04,4.0,NaN,1 minute blueberries cream i was craving blu...
146,416599,5,789314,2010-03-15,5.0,7.0,2 2 2 diet mocha while trying to come up with ...


In [42]:
recipes_2010.full_description

52       just peachy  cobbler all i can say is yummmmmm...
68       the heat  spicy party mix a spicy chex mix tha...
81       iowa state fair  sweet dough caramel cinnamon ...
104      1 minute blueberries   cream i was craving blu...
146      2 2 2 diet mocha while trying to come up with ...
                               ...                        
29897    zoe s chicken tarragon from a good housekeepin...
29907    zucchini and noodle slice a yummy, tasty slice...
29915    zucchini bread   bread machine originally from...
29926    zucchini chip cupcakes this is a great tasting...
29992    zucchini  courgette soup  good for weight watc...
Name: full_description, Length: 1538, dtype: object

1\.1 С использованием метода `DataFrame.iterrows` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - нахождение полного описания рецепта осуществляется внутри цикла по `iterrows` для каждой строки по отдельности.

In [130]:
def get_mean_len_A(df: pd.DataFrame) -> float:
    mask = []
    for _, row in df.iterrows():
        full = row['name'] + ' ' + row['description']
        mask.append(len(full))
    return np.mean(mask)

1\.2. С использованием метода `DataFrame.apply` таблицы:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - вызываете метод apply у таблицы, в качестве аргумента передаете функцию, которая возвращает полное описание для каждой строки;
    
    - считаете среднюю длину описаний, вызвав соответствующий метод серии.

In [132]:
def get_mean_len_B(df: pd.DataFrame) -> float:
    return np.mean(df.apply(lambda x: len(x['name'] + ' ' + x['description']), axis = 1))

1\.3. С использованием векторизованных методов серий `pd.Series`:

    - функция принимает на вход таблицу, содержащую рецепты за 2010 год;
    
    - при помощи векторизированных операций получаете столбец с полным описанием;
    
    - при помощи векторизированных операций получаете длины полного описания;
    
    - при помощи метода серий получаете среднюю длину полных описаний. 

In [134]:
def get_mean_len_C(df: pd.DataFrame) -> float:
    return (df['description'] + ' ' + df['name']).str.len().mean()

1.4 Проверьте, что результаты работы всех написанных функций корректны и совпадают. Измерьте выполнения всех написанных функций при помощи магических команд `time` и `timeit`.

In [139]:
%%time
get_mean_len_A(recipes)

CPU times: user 90.6 ms, sys: 3.66 ms, total: 94.2 ms
Wall time: 91.6 ms


265.501300390117

In [140]:
%%timeit
get_mean_len_A(recipes)

57.3 ms ± 3.36 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [141]:
%%time
get_mean_len_B(recipes)

CPU times: user 21.6 ms, sys: 1.76 ms, total: 23.4 ms
Wall time: 22.3 ms


265.501300390117

In [142]:
%%timeit
get_mean_len_B(recipes)

19.2 ms ± 580 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [143]:
%%time
get_mean_len_C(recipes)

CPU times: user 2.31 ms, sys: 289 µs, total: 2.6 ms
Wall time: 2.41 ms


265.501300390117

In [144]:
%%timeit
get_mean_len_C(recipes)

1.53 ms ± 101 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [145]:
get_mean_len_A(recipes) == get_mean_len_B(recipes) == get_mean_len_C(recipes)

True

## Анализ пошагового выполнения кода 

Вам предлагается воспользоваться функцией, которая собирает статистику о том, сколько отзывов содержат то или иное слово. 

In [71]:
import re


def get_word_reviews_count(df):
    word_reviews = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        recipe_id, review = row["recipe_id"], row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            if word not in word_reviews:
                word_reviews[word] = []
            word_reviews[word].append(recipe_id)

    word_reviews_count = {}
    for _, row in df.dropna(subset=["review"]).iterrows():
        review = row["review"]
        words = re.sub("[^A-Za-z\s]", "", review).split(" ")
        for word in words:
            word_reviews_count[word] = len(word_reviews[word])
    return word_reviews_count

2.1 Найдите узкие места в коде, проанализировав код функции по шагам, используя профайлер. Сохраните результаты работы профайлера в отдельную текстовую ячейку. Выпишите (словами), что в имеющемся коде реализовано неоптимально. 

In [72]:
%load_ext line_profiler 

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [73]:
%lprun -f get_word_reviews_count get_word_reviews_count(reviews)

Line #      Hits         Time  Per Hit   % Time  Line Contents
==============================================================
     4                                           def get_word_reviews_count(df):
     5         1          9.0      9.0      0.0      word_reviews = {}
     6    126680   12416124.0     98.0     22.5      for _, row in df.dropna(subset=["review"]).iterrows():
     7    126679    3666269.0     28.9      6.6          recipe_id, review = row["recipe_id"], row["review"]
     8    126679    1644083.0     13.0      3.0          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
     9   6918689    2703307.0      0.4      4.9          for word in words:
    10   6792010    3714115.0      0.5      6.7              if word not in word_reviews:
    11     93059      60229.0      0.6      0.1                  word_reviews[word] = []
    12   6792010    4149049.0      0.6      7.5              word_reviews[word].append(recipe_id)
    13                                           
    14         1          3.0      3.0      0.0      word_reviews_count = {}
    15    126680   13297198.0    105.0     24.1      for _, row in df.dropna(subset=["review"]).iterrows():
    16    126679    2393596.0     18.9      4.3          review = row["review"]
    17    126679    1842812.0     14.5      3.3          words = re.sub("[^A-Za-z\s]", "", review).split(" ")
    18   6918689    3093835.0      0.4      5.6          for word in words:
    19   6792010    6265352.0      0.9     11.3              word_reviews_count[word] = len(word_reviews[word])
    20         1          1.0      1.0      0.0      return word_reviews_count

In [ ]:
#ну самое ужасное это iterrows конечно
#множество ненужных циклов
#но сейчас подправим!

2.2  Оптимизируйте функцию и добейтесь значительного (как минимум, в 5 раз) прироста в скорости выполнения. Для демонстрации результата измерьте скорость выполнения оригинальной функции и функции, написанной вами.

In [93]:
%%timeit
get_word_reviews_count(reviews)

17.8 s ± 1.38 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [77]:
from collections import Counter

In [90]:
def my_get_word_reviews_count(df):
    
    text = ' '.join(reviews.review.dropna())
    words = re.sub("[^A-Za-z\s]", "", text).split()

    return Counter(words)

In [94]:
%%timeit
my_get_word_reviews_count(reviews)

2.05 s ± 45.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


## Numba

В файле `rating_predictions.json` хранятся данные о рейтингах рецептов и прогнозных значениях рейтингов для этого рецепта, полученных при помощи модели машинного обучения. 

Напишите несколько версий функции (см. [MAPE](https://en.wikipedia.org/wiki/Mean_absolute_percentage_error)) для расчета среднего абсолютного процентного отклонения значения рейтинга отзыва на рецепт от прогнозного значения рейтинга для данного рецепта. 


Замечание 1: в формуле MAPE под $A_t$ понимается рейтинг из отзыва $t$, под $F_t$ - прогнозное значения рейтинга отзыва $t$.

Замечание 2: в результате работы функций должно получиться одно число - MAPE для всего набора данных.

In [2]:
import json

In [3]:
with open(
    r'rating_predictions.json',
    'r',
    encoding = 'utf-8'    
) as fp:
    book = json.load(fp)

In [28]:
A_list = []
F_list = []

for rating in book:
    A_list.append(rating['rating'])
    F_list.append(rating['prediction'])

3\.1 Создайте два списка `A_list` и `F_list` на основе файла `rating_predictions.json`. Напишите функцию `mape_lists` без использования векторизованных операций и методов массивов `numpy` и без использования `numba` (проитерируйтесь по спискам и вычислите суммарное значение MAPE для всех элементов, а потом усредните результат).

Измерьте время выполнения данной функции на входных данных `A_list` и `F_list`. Временем, затрачиваемым на создание списков, можно пренебречь.
    

In [16]:
def mape_lists(A_list, F_list):
    S = 0
    for i in range(len(A_list)):
        S += abs((A_list[i]-F_list[i])/A_list[i])
    return 100 * S/len(A_list)

In [21]:
%%time
mape_lists(A_list, F_list)

CPU times: user 81.6 ms, sys: 3.55 ms, total: 85.1 ms
Wall time: 83.5 ms


13.325265503992638

In [39]:
%%timeit
mape_lists(A_list, F_list)

21.1 ms ± 900 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


3\.2. Создайте массивы `numpy` `A_array` и `F_array` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numpy` с использованием векторизованных операций и методов массивов `numpy`.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`. Временем, затрачиваемым на создание массивов, можно пренебречь.

In [19]:
import numpy as np

In [29]:
A_array = np.array(A_list)
F_array = np.array(F_list)

In [30]:
def mape_numpy(A_list, F_list):
    return 100/len(A_list) * sum(abs((A_list - F_list)/A_list))

In [31]:
%%time
mape_numpy(A_array, F_array)

CPU times: user 15.8 ms, sys: 3.3 ms, total: 19.1 ms
Wall time: 15.8 ms


13.325265503992638

In [40]:
%%timeit
mape_numpy(A_array, F_array)

10 ms ± 803 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


3\.3. Создайте объекты `numba.typed.List` `A_typed` и `F_typed` на основе списков `A_list` и `F_list`. Напишите функцию `mape_numba` без использования векторизованных операций и методов массивов `numpy`, но с использованием `numba`. 

Измерьте время выполнения данной функции на входных данных `A_typed` и `F_typed`. Временем, затрачиваемым на создание объектов `numba.typed.List`, можно пренебречь.

Измерьте время выполнения данной функции на входных данных `A_array` и `F_array`.

In [35]:
A_typed = numba.typed.List(A_list)
F_typed = numba.typed.List(F_list)

In [36]:
@njit
def mape_numba(A_list, F_list):
    S = 0    
    for i in range(len(A_list)):
        S += abs((A_list[i]-F_list[i])/A_list[i])
    return 100 * S/len(A_list)

In [38]:
%%time
mape_numba(A_typed, F_typed)

CPU times: user 226 ms, sys: 6.62 ms, total: 233 ms
Wall time: 232 ms


13.325265503992638

In [41]:
%%timeit
mape_numba(A_typed, F_typed)

1.16 ms ± 42.4 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


## Векторизация

Сайт-агрегатор устроил акцию: он дарит купоны на посещение ресторана тем пользователям, оставившим отзывы, идентификатор которых является _красивым числом_. Натуральное число называется _красивым_, если первая цифра числа совпадает с последней цифрой числа. 



4\.1 Напишите функцию `is_pretty`, которая для каждого идентификатора пользователя из файла определяет, получит ли он подарок. Запрещается преобразовывать идентификатор пользователя к строке. Подтвердите корректность реализации, продемонстрировав примеры.

In [58]:
import math

In [46]:
ids = reviews["recipe_id"].values

In [96]:
def is_pretty(n: int) -> bool:
    last = n % 10
    first = n // 10 ** int(math.log10(n)) #факт, math быстрее чем np
    return last == first

In [79]:
is_pretty(454)

True

In [80]:
is_pretty(3)

True

In [81]:
is_pretty(12345678)

False

4\.2 Посчитайте с помощью функции `is_pretty` количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.

In [89]:
sum([is_pretty(i) for i in ids])

11955

In [90]:
%%time
sum([is_pretty(i) for i in ids])

CPU times: user 352 ms, sys: 2.17 ms, total: 354 ms
Wall time: 356 ms


11955

In [97]:
%%timeit
sum([is_pretty(i) for i in ids])

328 ms ± 7.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


4\.3. При помощи `numpy` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [110]:
def my_vector_is_pretty(ids):
    return np.sum(ids % 10 == ids // 10 ** np.log10(ids).astype(int))

In [111]:
my_vector_is_pretty(ids)

11955

In [113]:
%%time
my_vector_is_pretty(ids)

CPU times: user 7.05 ms, sys: 2.67 ms, total: 9.71 ms
Wall time: 5.85 ms


11955

In [112]:
%%timeit
my_vector_is_pretty(ids)

2.48 ms ± 123 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


4\.4. При помощи `numba` создайте векторизованную версию функции `is_pretty`. Посчитайте с помощью этой векторизованной функции количество пользователей, которые получат подарок. Выведите это количество на экран. Измерьте время расчетов для входных данных `ids`.


In [118]:
numba_is_pretty = numba.vectorize(['int64(int64)'])(is_pretty)
numba_is_pretty(ids).sum()

11955

In [120]:
%%time
numba_is_pretty(ids).sum()

CPU times: user 6.43 ms, sys: 2.67 ms, total: 9.1 ms
Wall time: 5.2 ms


11955

In [119]:
%%timeit
numba_is_pretty(ids).sum()

2.27 ms ± 40.1 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
